In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("dschallenge.csv")
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


In [5]:
df.shape

(5000, 7)

In [6]:
df.isna().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

In [7]:
df.shop_id.nunique()

100

In [8]:
df.user_id.nunique()

301

In [14]:
shop_order_df = df[['shop_id','order_amount']].groupby('shop_id',sort=False).sum()
print(shop_order_df['order_amount'].sum() == df['order_amount'].sum())
shop_order_df


True


,order_amount
shop_id,
53,14560
92,6840
44,10224
18,17472
58,15042
...,...
90,19758
94,13400
46,14940


In [15]:
shop_item_df = df[['shop_id','total_items']].groupby('shop_id',sort=False).sum()
print(shop_item_df.total_items.sum() == df.total_items.sum())
shop_item_df

True


,total_items
shop_id,
53,130
92,76
44,71
18,112
58,109
...,...
90,111
94,100
46,90
